## Get Download URLs from SARVIEWS

In order to get a Download URL for a specific granule in SARVIEWS, you will need two things:
1. usgs_event_id
2. granule_name

In [1]:
import os
import json
import requests

In [2]:
usgs_event_id = 'us6000jrlp'
granule_name  = 'S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603'
product_name  = granule_name + '.zip'

First, you will need to make a GET request to https://gm3385dq6j.execute-api.us-west-2.amazonaws.com/events in order to retrieve a list of all of the events known to SARVIEWS. The response will be a list of the form:
```json
[
    {
        "depth": "",
        "description": "",
        "event_date": "",
        "event_id": "99",
        "event_type": "quake",
        "magnitude": ,
        "processing_timeframe": {
            "end": "",
            "start": ""
        },
        "usgs_event_id": "ak20076877",
        "wkt": ""
    },
    ...
]
```

In [3]:
event_list_res = requests.get('https://gm3385dq6j.execute-api.us-west-2.amazonaws.com/events')
event_list_res.status_code

200

Once we have the list of events, we can search through it to find the event with our desired usgs_event_id. Once we have the correct event, we can grab the event_id. This event_id is used as the query to find the list of downloadable products.

In [4]:
event_list = event_list_res.json()
event_obj  = next((item for item in event_list if ("usgs_event_id" in item) and (item["usgs_event_id"] == usgs_event_id)), None)
event_id   = event_obj['event_id']
print(event_id)

913a4ab7-704e-4aca-b971-40706161848c


Now that we have the event_id, we can make a GET request to https://gm3385dq6j.execute-api.us-west-2.amazonaws.com/events/{event_id}. This will respond all of the information associated with the event, including all of the products. The response will be of the form:
```json
{
    "depth": "",
    "description": "",
    "event_date": "",
    "event_id": "99",
    "event_type": "quake",
    "magnitude": 6.5,
    "processing_timeframe": {
        "end": "",
        "start": ""
    },
    "products": [
        {
            "event_id": "99",
            "files": {
                "browse_url": "",
                "product_name": "S1AA_20180504T161046_20180528T161048_VVP024_INT80_G_weF_5E24.zip",
                "product_size": 214683242,
                "product_url": "https://.../..._5E24.zip", <- DOWNLOAD LINK
                "thumbnail_url": ""
            },
            "granules": [...],
            "job_type": "",
            "processing_date": "",
            "product_id": "",
            "status_code": ""
        },
        ...
    ],
    "usgs_event_id": "ak20076877",
    "wkt": ""
}
```

In [5]:
event_get_res = requests.get(f'https://gm3385dq6j.execute-api.us-west-2.amazonaws.com/events/{event_id}')
event_get_res.status_code

200

Then, we can search through the response object's "products" for one whose "files" have a "product_name" that is the same as our desired granule name - sans the product_name's '.zip'.

In [6]:
event_get_list = event_get_res.json()
event_obj      = next((item for item in event_get_list["products"] if item["files"]["product_name"] == product_name), None)

Finally, we now have the correct product, and we can grab the "product_url" from "files" and use it to download the whole product.

In [7]:
print(product_url := event_obj['files']['product_url'])

https://hyp3-event-monitoring-productbucket-1t80jdtrfscje.s3.amazonaws.com/913a4ab7-704e-4aca-b971-40706161848c/b750eaa2-09e7-4ffc-8055-6a54b84b7404/S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603.zip


In [8]:
os.system(f'wget --quiet {product_url}')
os.system(f'unzip -qq {product_name}')
os.system(f'ls {granule_name}')

S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603.README.md.txt
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603.txt
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_amp.tif
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_amp.tif.xml
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_color_phase.kmz
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_color_phase.png
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_color_phase.png.aux.xml
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_color_phase.png.xml
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_corr.tif
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_corr.tif.xml
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_lv_phi.tif
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_lv_phi.tif.xml
S1AA_20230126T212437_20230219T212436_VVR024_INT80_G_weF_3603_lv_theta.tif
S1AA_20230126T212437_20230219T21

0